In [1]:
import findspark
findspark.init()
from pyspark import *
from pyspark.sql import *
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, StringType
import os
from pyspark.sql.functions import collect_list
import pandas as pd
from pyspark.sql.functions import countDistinct
from pyspark.context import SparkContext

In [2]:
conf = SparkConf()
spark = SparkSession.builder.appName("PageRank").getOrCreate()
spark

In [3]:
dataframe=spark.read.options(delimiter="\t",header=True).csv("wikilink_graph.2005-03-01.csv")

In [4]:
out=dataframe.groupby("page_id_from").agg(collect_list("page_id_to").alias("out_links")).withColumnRenamed("page_id_from","page_ido")


In [5]:
idsOut=set(out.rdd.map(lambda x: x.page_ido).collect())
idsOrigin=set(dataframe.rdd.map(lambda x: x.page_id_to).collect())

In [6]:
difference = idsOrigin.difference(idsOut)
diff_rdd=spark.sparkContext.parallelize(difference)
difference=diff_rdd.map(lambda x: (str(x),[str(-1)])).collect()


In [7]:
diff_dataframe=spark.createDataFrame(difference,out.schema.names)
appended=out.union(diff_dataframe)



In [8]:
panda=appended.toPandas()

In [9]:
panda.to_csv("preprocessed.csv",index=False,header=False,sep="\t")